In [1]:
# modify these for your own computer
#repo_directory = '/Users/iaincarmichael/Dropbox/Research/law/law-net/'
repo_directory = '/Users/Michael/Documents/GitHub/law-net/'

#data_dir = '/Users/iaincarmichael/Documents/courtlistener/data/'
data_dir = '/Users/Michael/Desktop/network_data/'

import os
import numpy as np
import sys
import matplotlib.pyplot as plt



# graph package
import igraph as ig


# stat
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import mixture
from sklearn.feature_extraction.text import CountVectorizer


# our code
sys.path.append(repo_directory + 'code/')


sys.path.append(repo_directory + 'vertex_metrics_experiment/code/')
#from bag_of_words import load_tf_idf
from bag_of_words import * 

# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'
nlp_dir = subnet_dir + 'nlp/'
nlp_bow_dir = nlp_dir + 'bow/' #bag-of-words matrix (and other info, i.e. vocab)
nlp_sub_dir = nlp_dir + 'bow_tfidf/' #tfidf matrix (and other info, i.e. vocab) computed from bag-of-words matrix

#nlp_df_bow_dir = nlp_dir + 'bow_df/'
nlp_df_sub_dir = nlp_dir + 'bow_tfidf_df/'

# all the file paths for .txt files
file_paths = glob.glob(text_dir + '*.txt')


# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import re
import glob
import cPickle as pickle

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords


from text_normalization import *
from pipeline_helper_functions import save_sparse_csr, load_sparse_csr

# compute bag of words matrix (bow matrix)
min_df (0.3) and max_df (0.7)

In [3]:
%%time

## compute bag_of_words matrix (copied from bag_of_words.py)

#tf_iter = textfile_iter(file_paths)
text_data = []
for fle in file_paths:
    text_data.append(open(fle, "r").read())
        
print len(text_data)
        


# stemmer and stop words
stemmer = PorterStemmer()
stop_words = stopwords.words("english")
min_df = 0.3  # be careful about integer/float, see sklean documentation
max_df = 0.7

# stem stop words
stop_words = [stemmer.stem(w) for w in stop_words]

bag_of_words = CountVectorizer(min_df=min_df,
                               max_df=max_df,
                               tokenizer=StemTokenizer(stemmer),
                               stop_words=stop_words)

#bow_matrix = bag_of_words.fit_transform(file_paths)
bow_matrix = bag_of_words.fit_transform(text_data)

27885
Wall time: 1h 6min 3s


In [4]:
bow_matrix

<27885x488 sparse matrix of type '<type 'numpy.int64'>'
	with 6029712 stored elements in Compressed Sparse Row format>

# transform the bow matrix to tf-idf matrix using 'TfidfTransformer'

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer

In [6]:
%%time
# compute tfidf matrix from bag of words

tfidf = TfidfTransformer()

tfidf_matrix = tfidf.fit_transform(bow_matrix)

Wall time: 280 ms


In [7]:
tfidf_matrix

<27885x488 sparse matrix of type '<type 'numpy.float64'>'
	with 6029712 stored elements in Compressed Sparse Row format>

# store tfidf matrix information (vocab, matrix, row indices) in new directory: 'nlp_df_sub_dir'

In [8]:
if not os.path.exists(nlp_df_sub_dir):
    os.mkdir(nlp_df_sub_dir)

# is it ok to save the 'vocab' from 'bag_of_words.get_feature_names()'?

In [10]:
# vocabulary
vocab = bag_of_words.get_feature_names()

# map from CL opinion ids to bow matix index
op_id_to_bow_id = {re.search(r'(\d+)\.txt', file_paths[i]).group(1): i for
                   i in range(len(file_paths))}



# save data
save_sparse_csr(nlp_df_sub_dir + 'tfidf_matrix', tfidf_matrix)

# saved row indices
with open(nlp_df_sub_dir + 'op_id_to_bow_id.p', 'wb') as fp:
    pickle.dump(op_id_to_bow_id, fp)

# save vocab
with open(nlp_df_sub_dir + 'vocab.p', 'wb') as fp:
    pickle.dump(vocab, fp)

In [12]:
vocab

[u'15',
 u'16',
 u'17',
 u'18',
 u'19',
 u'20',
 u'21',
 u'22',
 u'23',
 u'24',
 u'25',
 u'26',
 u'27',
 u'28',
 u'29',
 u'30',
 u'31',
 u'32',
 u'33',
 u'34',
 u'35',
 u'36',
 u'37',
 u'38',
 u'39',
 u'40',
 u'41',
 u'42',
 u'43',
 u'44',
 u'45',
 u'46',
 u'47',
 u'48',
 u'49',
 u'50',
 u'52',
 u'60',
 u'absenc',
 u'accept',
 u'accord',
 u'accordingli',
 u'account',
 u'action',
 u'actual',
 u'ad',
 u'addit',
 u'administr',
 u'admit',
 u'adopt',
 u'affect',
 u'agre',
 u'al',
 u'alleg',
 u'allow',
 u'alon',
 u'alreadi',
 u'although',
 u'amend',
 u'among',
 u'amount',
 u'anoth',
 u'answer',
 u'appar',
 u'appear',
 u'appel',
 u'appli',
 u'applic',
 u'appropri',
 u'approv',
 u'april',
 u'argu',
 u'argument',
 u'aris',
 u'ask',
 u'assert',
 u'assign',
 u'assum',
 u'attempt',
 u'attorney',
 u'avail',
 u'b',
 u'bar',
 u'base',
 u'basi',
 u'bear',
 u'becam',
 u'becom',
 u'believ',
 u'benefit',
 u'beyond',
 u'bill',
 u'board',
 u'bound',
 u'brief',
 u'bring',
 u'brought',
 u'busi',
 u'c',
 u'ca